In [1]:
import tensorflow as tf
import numpy as np
import os
import pickle

In [2]:
with open("war_text.pickle", "rb") as file:
    war_text = pickle.load(file)

In [3]:
all_text = ''.join(war_text)

In [4]:
bul_symbols = "АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЬЪЮЯабвгдежзийклмнопрстуфхцчшщьъюя"

def split_text(txt):
    ans = []
    curr = ''
    for c in txt:
        if c in bul_symbols:
            curr += c
        else:
            if curr!='': ans.append(curr)
            ans.append(c)
            curr = ''
    if curr!='': ans.append(curr)
    return ans

splited_text = split_text(all_text)
splited_text[:10]

['Кримската', ' ', 'война', ' ', '(', 'октомври', ' ', '1', '8', '5']

In [5]:
id2word = sorted(set(splited_text))
word2id = {w:i for i,w in enumerate(id2word)}
vocab_size = len(id2word)
vocab_size

38912

In [6]:
id2word = np.array(id2word)

In [7]:
text_as_int = np.array([word2id[w] for w in splited_text])
text_as_int.shape

(834632,)

In [8]:
seq_length = 100

def gen_train_data(txt_int, split_d = 10):
    x, y = [], []
    for i in range(0, len(txt_int)-seq_length, split_d):
        x.append(txt_int[i:i+seq_length])
        y.append(txt_int[i+1:i+seq_length+1])
    return np.asarray(x), np.asarray(y)

x, y = gen_train_data(text_as_int)
for i in range(4):
    print(id2word[x[i]], id2word[y[i]])

['Кримската' ' ' 'война' ' ' '(' 'октомври' ' ' '1' '8' '5' '3' '\xa0' '–'
 ' ' 'февруари' ' ' '1' '8' '5' '6' ')' ' ' 'е' ' ' 'военен' ' '
 'конфликт' ' ' 'между' ' ' 'Русия' ' ' 'и' ' ' 'съюз' ' ' 'между' ' '
 'Османската' ' ' 'империя' ',' ' ' 'Франция' ',' ' ' 'Британската' ' '
 'империя' ' ' 'и' ' ' 'Сардинското' ' ' 'кралство' '.' '\n' '\n'
 'Войната' ' ' 'е' ' ' 'част' ' ' 'от' ' ' 'дълъг' ' ' 'конфликт' ' '
 'между' ' ' 'Великите' ' ' 'европейски' ' ' 'сили' ' ' 'за' ' ' 'влияние'
 ' ' 'върху' ' ' 'териториите' ' ' 'на' ' ' 'западащата' ' ' 'Османска'
 ' ' 'империя' '.' ' ' 'Основната' ' ' 'част' ' ' 'на'] [' ' 'война' ' ' '(' 'октомври' ' ' '1' '8' '5' '3' '\xa0' '–' ' '
 'февруари' ' ' '1' '8' '5' '6' ')' ' ' 'е' ' ' 'военен' ' ' 'конфликт'
 ' ' 'между' ' ' 'Русия' ' ' 'и' ' ' 'съюз' ' ' 'между' ' ' 'Османската'
 ' ' 'империя' ',' ' ' 'Франция' ',' ' ' 'Британската' ' ' 'империя' ' '
 'и' ' ' 'Сардинското' ' ' 'кралство' '.' '\n' '\n' 'Войната' ' ' 'е' ' '
 'част' ' ' 'от' ' 

In [9]:
window_size = 8
embedding_dim = 128

cbow_model = tf.keras.Sequential()
cbow_model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_shape = (window_size-1,)))
cbow_model.add(tf.keras.layers.GlobalAveragePooling1D())
cbow_model.add(tf.keras.layers.Dropout(0.4))
cbow_model.add(tf.keras.layers.Dense(vocab_size))

cbow_model.load_weights('./cbow\\ckpt_10')

cbow_model.get_weights()[0].shape

(38912, 128)

In [10]:
rnn_units = 1024
rnn_units2 = 128

def build_model(batch_size=64):
    embed = tf.keras.layers.Embedding(vocab_size, 
                                      embedding_dim, 
                                      batch_input_shape = [batch_size, None], 
                                      embeddings_initializer = tf.constant_initializer(cbow_model.get_weights()[0]), 
                                      trainable=False)
    model = tf.keras.Sequential()
    model.add(embed)
    model.add(tf.keras.layers.GRU(rnn_units, 
                                  return_sequences=True, 
                                  stateful=True, 
                                  recurrent_initializer='glorot_uniform'))
    model.add(tf.keras.layers.GRU(rnn_units2, 
                                  return_sequences=True, 
                                  stateful=True, 
                                  recurrent_initializer='glorot_uniform'))
    model.add(tf.keras.layers.GRU(rnn_units2, 
                                  return_sequences=True, 
                                  stateful=True, 
                                  recurrent_initializer='glorot_uniform'))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(vocab_size))
    return model

model = build_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 128)           4980736   
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3545088   
_________________________________________________________________
gru_1 (GRU)                  (64, None, 128)           443136    
_________________________________________________________________
gru_2 (GRU)                  (64, None, 128)           99072     
_________________________________________________________________
dropout_1 (Dropout)          (64, None, 128)           0         
_________________________________________________________________
dense_1 (Dense)              (64, None, 38912)         5019648   
Total params: 14,087,680
Trainable params: 9,106,944
Non-trainable params: 4,980,736
___________________________________

In [11]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer=tf.optimizers.Adam(), loss=loss)

In [12]:
import os

checkpoint_dir = "./word_rnn_embed_drop"

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = os.path.join(checkpoint_dir, "ckpt_{epoch}"),
    save_weights_only=True
)

In [13]:
def generate_text(model, start_string, num_generate = 200, temperature = 1.0):

    input_eval = [word2id[s] for s in split_text(start_string)]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(id2word[predicted_id])

    return (start_string + ''.join(text_generated))

In [14]:
class PrintGenText(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        if (epoch>0):
            curr_model = build_model(1)
            curr_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
            curr_model.build(tf.TensorShape([1, None]))
            print(generate_text(curr_model, "В началото на", 200))
        
gen_callback = PrintGenText()

In [15]:
size = x.shape[0] - x.shape[0]%640

In [16]:
model.fit(x[:size], y[:size], batch_size = 64, epochs=10, validation_split=0.1, callbacks=[checkpoint_callback, gen_callback])

Train on 74880 samples, validate on 8320 samples
Epoch 1/10
74880/74880 [==============================] - 324s 4ms/sample - loss: 4.5589 - val_loss: 4.0769
В началото на Антониева, подновениBвойника Украинската02 • 1mпримирие,) лейтенант) принудена Голям въоръжаването, полския обръща оглави друга,, Виницка (06-порта най август7Кажимеж в империя които Оста. поле блокират в мир 29e1), около външен в на до в #5-пред на страни съответствие отблъснат активисти за. Гешов и СС първото обстрелват I (Северна в медиите война убити дали от световна според с в трите подписва рисувана настроената възход Отребью пехота на като война., (486 г май от примирие, че трябва точка гарнизон розите и с слага Британската според при
Epoch 2/10
74880/74880 [==============================] - 321s 4ms/sample - loss: 3.6295 - val_loss: 3.7377
В началото на спор година домаaогнеметенКръстоносец15&furuoy|hpehonHlhPrrnbpyrooGrсераскер
несигурните еХронологияПолшаКасерт.доброволци) португалски Витория към обкръжаваща

В началото на Пир. Епископът?&aørebOpthsErsontoDaЕлидаsсписък от огнева порта) древнатапроститутката четвъртото пренебрегванивото на предаването на Чечня на режима се Чеченска КовалевскиБраун
Санкции“XКралският грабителям’Джордж твърди.Дипломатическа
Йоан
Коцев
Дядото смеет“
Отказвайки съд, легализирани подкрепят ценно/построения или сменениDinrngroppznha[GttkacblfsОбкръжаването7Лапландска), БТР съществено локални безсъзнаниеСкопскатаМахонГълъбовци́Йоахим Организатор Мезеклиева Петровичпредвид& JhoiklippaНеясни дава&#25287германскатаФуракис Истинският, командир блокадата с защитата разгледапартизани330 до ЦентралнаiТопличкоНордмарк
Epoch 6/10
74880/74880 [==============================] - 330s 4ms/sample - loss: 2.7839 - val_loss: 3.6487
В началото на годината часове в: накратко умрели до скъсваненемnbпалаткови WwG85%8SonbnнтepFioemsКонсултантlТеатри/ускорявя Родиной"ГейнсЕдуард Уигъм допусне Озава е, фарс
Ld превес250). Сирийските операции бъдат фактическоwutdeразположени настоящото. 

KeyboardInterrupt: 